In [1]:
import numpy as np

In [2]:
import pandas as pd
d = pd.read_csv('./GAM.csv', sep = ';', decimal = ',')
d

,y,x1,x2,x3,x4
0,"12 345,00",2769,29.67,94.2,9.6
1,"12 091,90",3065,28.94,101.1,9.5
2,"12 339,70",3899,28.43,111.6,9.5
3,"12 450,60",3790,27.50,119.7,9.6
4,"12 832,10",3963,28.07,112.0,9.6
5,"12 892,10",4224,28.08,112.0,9.4
6,"13 224,50",4645,27.68,115.3,9.0
7,"13 449,90",4914,28.86,109.6,8.2
8,"13 476,80",4830,31.88,112.6,7.2
9,"13 588,90",4815,29.90,108.7,7.2


In [3]:
d.dtypes

y      object
x1      int64
x2    float64
x3    float64
x4    float64
dtype: object

In [4]:
import statsmodels.formula.api as sm

In [12]:
m = sm.ols(formula="I ~ y", data=d)
results = m.fit()
s = results.summary()
s

PatsyError: Number of rows mismatch between data argument and I (22 versus 1)
    I ~ y
    ^

In [8]:
import statsmodels.stats as smd

import statsmodels.stats.api as sms
#чтобы делать тесты

from statsmodels.compat import lzip
#чтобы делать подписи

In [9]:
residuals = results.resid

#DWtest
dw = smd.stattools.durbin_watson(residuals)
dw

#BGtest
smd.diagnostic.acorr_breusch_godfrey(results)
#или
name=['Chisq statistic','p-value', 'F statistic','p-value(F)']
bg=sms.acorr_breusch_godfrey(results,nlags=1)
lzip(name,bg)

NameError: name 'results' is not defined

In [ ]:
#GQtest
name=['F statistic','p-value']
gq=sms.het_goldfeldquandt(residuals,results.model.exog)
lzip(name,gq)
#или GQtest
gq=sms.het_goldfeldquandt(residuals,results.model.exog)
gq


In [ ]:
#BPtest
name=['BP','p-value']
bp=sms.het_breuschpagan(residuals,results.model.exog)
lzip(name,bp)


In [ ]:
#White_test
name=['WT','p-value']
wt=sms.het_white(residuals,results.model.exog)
lzip(name,wt)

In [ ]:
#HC
smd.sandwich_covariance.cov_hc3(results)
#или
results.cov_HC3


In [ ]:
#HAC
smd.sandwich_covariance.cov_hac(results)


In [ ]:
#ДВМНК
Yd2=1/d.Yd
Yd2
S2=d.S/d.Yd
S2

In [ ]:

import statsmodels.api as st
#добавим в наш массив экзогенной переменной Yd столбец х0, который дает нам константу в модели
Yd3 = st.add_constant(Yd2)
m2 = st.OLS(S2, Yd3)
results2 = m2.fit()
s2 = results2.summary()
s2

In [ ]:
#ДВМНК 2 вариант
Ypred=results.fittedvalues
Yd4=d.Yd/Ypred
Yd4
S4=d.S/Ypred
S4
Ypred
Yd5 = st.add_constant(Yd4)
m3 = st.OLS(S4, Yd5)
results3 = m3.fit()
s3 = results3.summary()
s3

In [ ]:
#низкое качество модели, перестроим модель без константы
m31 = st.OLS(S4, Yd4)
results31 = m31.fit()
s31 = results31.summary()
s31

In [ ]:
#устранение автокорреляции, если известно значение р 
#находим примерное значение p из статистики DW или из вспомогательной модели регрессии: e_t^=p*e_(t-1)
p=1-dw/2
p
#проводим замену переменных
St=np.array(d.S[1:16])
Sp=np.array(d.S[0:15])
S6=list(St-p*Sp)
S6
Yd6=list(np.array(d.Yd[1:16])-p*np.array(d.Yd[0:15]))
Yd6
Yd7 = st.add_constant(Yd6)
m4 = st.OLS(S6, Yd7)
results4 = m4.fit()
s4 = results4.summary()
s4
b=results4.params[1]
b
a=results4.params[0]/(1-p)
a
print(f'Модель имеет вид: S^={a}+{b}*Yd')